In [23]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [24]:
if 'data' not in pn.state.cache.keys():
    df = pd.read_csv('https://raw.githubusercontent.com/WillBallentine/cov_dashboard/main/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time_-_ARCHIVED.csv')
    pn.state.cache['data'] = df.copy()
    
else:
    df = pn.state.cache['data']

In [25]:
df

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,NaN,NaN,0,0.0,03/12/2021 03:20:13 PM,Agree,NaN
1,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,NaN,NaN,9,0.0,12/02/2021 02:35:20 PM,Agree,Not agree
2,01/02/2022,AS,11,NaN,NaN,0,0.0,0,NaN,NaN,0,0.0,01/03/2022 03:18:16 PM,NaN,NaN
3,11/22/2021,AL,841461,620483.0,220978.0,703,357.0,16377,12727.0,3650.0,7,3.0,11/22/2021 12:00:00 AM,Agree,Agree
4,05/30/2022,AK,251425,NaN,NaN,0,0.0,1252,NaN,NaN,0,0.0,05/31/2022 01:20:20 PM,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60055,02/09/2021,TX,2504556,NaN,NaN,13329,2676.0,43306,NaN,NaN,207,0.0,02/11/2021 12:00:00 AM,Not agree,Not agree
60056,11/20/2020,FL,913561,NaN,NaN,8217,1677.0,19014,NaN,NaN,79,5.0,11/20/2020 12:00:00 AM,Not agree,Not agree
60057,08/17/2020,NM,23500,NaN,NaN,92,0.0,682,NaN,NaN,4,0.0,08/19/2020 12:00:00 AM,NaN,Not agree
60058,06/17/2020,MS,24223,24038.0,185.0,521,6.0,1191,1172.0,19.0,9,0.0,06/19/2020 12:00:00 AM,Agree,Agree


In [35]:
df = df.fillna(0)

In [36]:
idf = df.interactive()

In [37]:
#Define panel widgets
state_select = pn.widgets.RadioBoxGroup(name="State Select", options=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY',], inline=False)
state_select

RadioBoxGroup(name='State Select', options=['AL', 'AK', 'AZ', ...], value='AL')

In [40]:
state_pipeline = (idf[(idf.state) == state_select] .groupby(['tot_cases']))

In [41]:
state_pipeline

In [42]:
tot_plot = state_pipeline.hvplot(x='submission_date', by='state', y='tot_cases', title="Total cases by state")
tot_plot

AttributeError: ((dim('*', getitem, dim('*').pd.state)==RadioBoxGroup(name='State Select', options=['AL', 'AK', 'AZ', ...], value='AL')).groupby(['tot_cases'])).hvplot(by='state', title='Total cases by state', x='submission_date', y='tot_cases') could not be applied to ':Dataset   [submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths]', 'hvplot' method does not exist on DataFrameGroupBy type.